Goal: regressions that work with deductibles (which might be overly amibtious). 

Basically, we have a claim count, which is a Poisson model. Every claim will come from a gamma distribtuion. However, if it is under a certain deductible, we do not code it in. 

Couple of imagined steps: 
- do a Poisson, where claim count is generated randomly, but every claim is ignored with X% probability (this is probably super easy)
- same as above, but make the X% probability depend on a variable, e.g., for every 1k increase, the chance of observing decreas
- make a Gamma with deductible
- combine everything, features determine baseline frequency, then the severity is clipped by a deductible amount, which then might impact frequency

Found some stuff like this one: https://www.stata.com/stata14/censored-poisson/ but it's not the type of censored I am looking for. 

Maybe censored is the wrong word...

In [1]:
import numpy as np
import pandas as pd

In [10]:
%load_ext autoreload
%autoreload 2

# Frequency with Fixed Random Claim Cancel

One binary categorical feature, and each claim is cancelled by a fix percentage. The cancelled ones we don't observe. 

My theory is that this is simply going to be another Poisson where we multiply the lambda with the prob of cancel. Or 1 - prob of cancel...

In [2]:
# parameters
cancel_prob = 0.3 # each claim has a chance to not materialise

In [3]:
df = pd.DataFrame.from_dict({'cat_feature' : np.repeat([0,1],100000)})
df['poisson_lambda'] = df.apply(lambda x: 0.2 if x['cat_feature'] == 0 else 0.7, axis = 1)
df['raw_claim_number'] = np.random.poisson(df['poisson_lambda'])

Every raw_claim_number can be cancelled with the same chance. I think I want to generate this from scratch... If a row has a positive raw_claim_number, we want to generate that many random uniform variables, and put them in a list. And those lists will be in the next column. 

In [4]:
df['claim_cancellation'] = df.apply(lambda x: list(np.random.uniform(0,1,int(x['raw_claim_number']))), axis = 1)
df['claim_number'] = df.apply(lambda x: np.sum(cancel_prob <= np.array(x['claim_cancellation'])), axis = 1)

## Empirical Analysis

Did the censored claim count become Poisson with lambda that is original * chance of not canceling? 

In [5]:
empirical = pd.DataFrame(df[df['cat_feature'] == 1]['claim_number'].value_counts()).reset_index()
empirical['claim_number'] = empirical['claim_number'] / sum(empirical['claim_number'])
empirical.columns = ["value", "prob"]

In [11]:
from scipy.special import factorial # for elementwise factorial calc of array

from functions import poisson_likelihood

In [14]:
empirical['theoretical_prob'] = empirical.apply(
    lambda x: poisson_likelihood(x["value"], 0.7 * (1-cancel_prob)), axis = 1)

In [15]:
empirical

,value,prob,theoretical_prob
0,0,0.61557,0.612626
1,1,0.29879,0.300187
2,2,0.07267,0.073546
3,3,0.01143,0.012012
4,4,0.00143,0.001472
5,5,0.00010,0.000144
6,6,0.00001,0.000012


It doesn't look wildly off...

## Math

Proving the above formulaically. 

## GLM

Question: can we fit a GLM based on that feature? We want to start with an offset. 